In [1]:
import pandas as pd 
import os
meta_data = pd.read_csv(r'N:\CBISDDSM\calcTrain\metadatacsv.csv',delimiter=";")
calc_train=pd.read_csv(r'N:\CBISDDSM\calcTrain\calc_case_description_train_set.csv')
# DataFrame'deki 'image file path' sütununu bölme
calc_train[['path_part1', 'path_part2', 'path_part3']] = calc_train['image file path'].str.split('/', n=3, expand=True).iloc[:, :3]

#print(mass_train.loc[30, ['path_part1', 'path_part2', 'path_part3']])  
#print(mass_train.loc[31, ['path_part1', 'path_part2', 'path_part3']])  
# mass_train'e yeni bir sütun ekleyerek default olarak NaN atayalım
calc_train["File Location"] = None

# meta_data'daki ilgili sütunları seçelim
meta_data_subset = meta_data[["Subject ID", "Study UID", "Series UID", "File Location"]]

# mass_train ile meta_data'yı birleştir
merged_df = calc_train.merge(
    meta_data_subset,
    left_on=["path_part1", "path_part2", "path_part3"],
    right_on=["Subject ID", "Study UID", "Series UID"],
    how="left"  # mass_train'deki tüm satırları koruyalım
)

# Gereksiz tekrar eden sütunları (Subject ID, Study UID, Series UID) kaldıralım
merged_df.drop(columns=["Subject ID", "Study UID", "Series UID"], inplace=True)

# Yeni CSV dosyasını kaydet
merged_df.to_csv(r"N:\CBISDDSM\calcTrain\calc_train_updated.csv", index=False)

print("Yeni dosya kaydedildi: calc_train_updated.csv")

merged_df=pd.read_csv(r'N:\CBISDDSM\calcTrain\calc_train_updated.csv')

# Ana dizini belirt
print(merged_df["File Location_y"].isna().sum())  # NaN olanların sayısını gösterir
print(merged_df.loc[0, "File Location_y"])  # İlk satırdaki "File Location" değerini yazdır
base_path = r"N:\CBISDDSM\calcTrain"

# File Location_y sütunundaki yolları güncelle
merged_df["File Location_y"] = merged_df["File Location_y"].apply(lambda x: os.path.join(base_path, x.lstrip(".\\")))
merged_df["File Location_y"] = merged_df["File Location_y"].apply(lambda x: os.path.join(x, "1-1.dcm"))
# Güncellenmiş ilk birkaç satırı görelim
# "File Location_y" sütunundaki aynı dosya yolunu sadece bir kez tut ve diğerlerini sil
merged_df = merged_df.drop_duplicates(subset="File Location_y", keep="first").reset_index(drop=True)

merged_df.to_csv(r"N:\CBISDDSM\calcTrain\calc_train_updated.csv", index=False)

merged_df=pd.read_csv(r'N:\CBISDDSM\calcTrain\calc_train_updated.csv')
print(merged_df.loc[0, "File Location_y"])  # İlk satırdaki "File Location" değerini yazdır

Yeni dosya kaydedildi: calc_train_updated.csv
0
.\CBIS-DDSM\Calc-Training_P_00005_RIGHT_CC\08-07-2016-DDSM-NA-23157\1.000000-full mammogram images-38548
N:\CBISDDSM\calcTrain\CBIS-DDSM\Calc-Training_P_00005_RIGHT_CC\08-07-2016-DDSM-NA-23157\1.000000-full mammogram images-38548\1-1.dcm


In [2]:
merged_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1227 entries, 0 to 1226
Data columns (total 19 columns):
 #   Column                   Non-Null Count  Dtype  
---  ------                   --------------  -----  
 0   patient_id               1227 non-null   object 
 1   breast density           1227 non-null   int64  
 2   left or right breast     1227 non-null   object 
 3   image view               1227 non-null   object 
 4   abnormality id           1227 non-null   int64  
 5   abnormality type         1227 non-null   object 
 6   calc type                1219 non-null   object 
 7   calc distribution        1027 non-null   object 
 8   assessment               1227 non-null   int64  
 9   pathology                1227 non-null   object 
 10  subtlety                 1227 non-null   int64  
 11  image file path          1227 non-null   object 
 12  cropped image file path  1227 non-null   object 
 13  ROI mask file path       1227 non-null   object 
 14  path_part1              

In [3]:
import os
import pydicom
import cv2
from PIL import Image
import numpy as np
from tqdm import tqdm  # ilerleme çubuğu için

# merged_df DataFrame'inizdeki her satırı işleyelim
for idx, row in tqdm(merged_df.iterrows(), total=len(merged_df)):
    dcm_path = row["File Location_y"]
    png_path = os.path.join(os.path.dirname(dcm_path), "1-1.png")  # Aynı dizine .png olarak kaydediliyor

    try:
        # DICOM dosyasını oku
        dicom = pydicom.dcmread(dcm_path)
        pixel_array = dicom.pixel_array

        # Normalize et (0-255 arası)
        pixel_array = cv2.normalize(pixel_array, None, 0, 255, cv2.NORM_MINMAX)
        pixel_array = pixel_array.astype(np.uint8)

        # Orijinal görüntü boyutlarını al
        height, width = pixel_array.shape[:2]

        # %4 oranında kırpma değerlerini hesapla
        crop_x = int(width * 0.04)
        crop_y = int(height * 0.04)

        # Görüntüyü kırp: üst, alt, sol ve sağtan %4 içeri
        cropped_img = pixel_array[crop_y:height - crop_y, crop_x:width - crop_x]

        # Kırpılmış görüntüyü 256x256 boyutuna yeniden boyutlandır
        resized_img = cv2.resize(cropped_img, (256, 256))

        # Kaydet (grayscale PNG)
        Image.fromarray(resized_img).save(png_path)
    except Exception as e:
        print(f"Hata oluştu: {dcm_path} -> {e}")

100%|██████████| 1227/1227 [01:40<00:00, 12.22it/s]
